## Climbing Shoes Analysis Project

The primary goal of this project is to determine the most cost-efficient climbing shoes for an intermediate climber. My motivation for this project comes from both my love for climbing and my need for new climbing shoes. Shoes are such an important and powerful aspect of climbing, that they often make or break climbing attempts. I often joke that anytime I fail a climb, it's a problem with the shoes.

My projection for this project involves web-scraping information from the men's climbing shoes section from the REI (Recreational Equipment Inc.) website. Information I want to get include the shoe name, shoe brand, price, discount status, and ratings. Once I get this information, I want to utilize an SQL database to store the data and potentially create a prediction model that scores the shoes. Lastly, I want to summarize my findings in clean visualizations through Tableau and/or Python.

Let's get started! First things first, let's import some python packages import for web-scraping as well as SQL connecting.

In [127]:
# Import necessary packages for webscraping and data collection
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import matplotlib as plt
from splinter.browser import Browser
import numpy as np

# Load SQL extension
%load_ext sql

# Matplot frontend
%matplotlib inline

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Step One: Webscraping

I originally only wanted to use the requests and Beautiful Soup packages to webscrape the REI site, but the products' ratings were dynamic html content, which Beautiful Soup can't access. Thus, I delved into the splinter package specifically for the ratings. This wasn't really what I wanted because I'm unfamiliar with this process and also I needed to download chromedriver.exe which opens a new browser while running the program. In addition, having this additional software means that I need to eventually update these extra parts when Chrome or chromedriver updates rollout.

In [128]:
# Define the primary url
master_url = 'https://www.rei.com/c/mens-climbing-shoes?page=1'

# Splinter browser opens new chrome browser window
br = Browser('chrome')
br.visit(master_url)

# Make a request to the primary url and uses an html parser to gather data
req = requests.get(master_url)
soup = bs4.BeautifulSoup(req.text, 'html.parser')

In [142]:
# Create a dataframe to store data in
shoes_df = pd.DataFrame(columns = ['Shoe Name', 'Shoe Company', 'Current Price', 'Original Price', 'Discount'])

# Use find all to get a list of each product's html information
page_lst = soup.find_all('li', {'class': 'pPe0GNuagvmEFURs1Q_vm'})

# Create a for loop that runs through each product and uses the find function
# to aquire specific data observations
for element in page_lst:
    shoe_name = element.find('div', {'class': 'r9nAQ5Ik_3veCKyyZkP0b'}).text
    if shoe_name[-8:] == " - Men's":
        shoe_name = shoe_name[:-8]
    
    shoe_company = element.find('div', {'class': '_1fwp3k8dh1lbhAAenp87CH'}).text
    
    if element.find('span', {'class': '_2xZVXKL4Bd0pJyQCumYi9P'}) != None:
        current_price = element.find('span', {'class': '_2xZVXKL4Bd0pJyQCumYi9P'}).text
        original_price = element.find('span', {'class': 'Dm_X3ktyv_w_gPWuYlJf_'}).text
        original_price = original_price[11:]
    else:
        current_price = element.find('div', {'class': '_1zwqhlCzOK-xETXwFg_-iZ'}).text
        original_price = element.find('div', {'class': '_1zwqhlCzOK-xETXwFg_-iZ'}).text
    
    if current_price != original_price:
        on_sale = True
    else:
        on_sale = False
    
    # Append each row to the shoes dataframe
    temp_row = [shoe_name, shoe_company, current_price, original_price, on_sale]
    shoes_df.loc[len(shoes_df)] = temp_row

In [144]:
shoes_df.head()

,Shoe Name,Shoe Company,Current Price,Original Price,Discount
0,Rogue VCS Climbing Shoes,Five Ten,$39.93,$100.00,True
1,Quantum Climbing Shoes,Five Ten,$69.93,$185.00,True
2,Finale Climbing Shoes,La Sportiva,$109.00,$109.00,False
3,Momentum Climbing Shoes - Ash,Black Diamond,$94.95,$94.95,False
4,Instinct VS Climbing Shoes,Scarpa,$141.69,$189.00,True


In [164]:
# Use the splinter browser to gather dynamic html data of ratings
# Also clean the data as we scrape it
ratings = []
for item in br.find_by_tag('dt'):
    rating = item.text.strip()[6:]
    if (rating[0].isalnum() == False):
        rating = rating[6:]
        rating = rating.split('s')[0].strip()
    else:
        rating = np.nan
    ratings.append(rating)
shoes_df['Rating'] = ratings

In [165]:
# Let's look at the first 10 to see if the null values mapped properly
shoes_df.head(10)

,Shoe Name,Shoe Company,Current Price,Original Price,Discount,Rating,Rating Count
0,Rogue VCS Climbing Shoes,Five Ten,$39.93,$100.00,True,4.5 out of 5,6
1,Quantum Climbing Shoes,Five Ten,$69.93,$185.00,True,4.8 out of 5,5
2,Finale Climbing Shoes,La Sportiva,$109.00,$109.00,False,4.4 out of 5,72
3,Momentum Climbing Shoes - Ash,Black Diamond,$94.95,$94.95,False,4.5 out of 5,60
4,Instinct VS Climbing Shoes,Scarpa,$141.69,$189.00,True,4.5 out of 5,23
5,Origin Climbing Shoes,Scarpa,$74.19,$99.00,True,NaN,0
6,Focus Climbing Shoes,Black Diamond,$99.93,$179.95,True,3.8 out of 5,6
7,Momentum Climbing Shoes,Black Diamond,$46.73,$94.95,True,4.5 out of 5,4
8,Free Range Climbing Shoes,So iLL,$80.73,$149.00,True,4.2 out of 5,24
9,Arpia Climbing Shoes,Scarpa,$119.19,$159.00,True,5 out of 5,3


In [166]:
# Same as above, use splinter browser to gather dynamic html content
rating_counts = []
for item in br.find_by_tag('dd'):
    counts = item.text.strip('()')
    rating_counts.append(counts)
shoes_df['Rating Count'] = rating_counts

In [167]:
shoes_df.head(10)

,Shoe Name,Shoe Company,Current Price,Original Price,Discount,Rating,Rating Count
0,Rogue VCS Climbing Shoes,Five Ten,$39.93,$100.00,True,4.5 out of 5,6
1,Quantum Climbing Shoes,Five Ten,$69.93,$185.00,True,4.8 out of 5,5
2,Finale Climbing Shoes,La Sportiva,$109.00,$109.00,False,4.4 out of 5,72
3,Momentum Climbing Shoes - Ash,Black Diamond,$94.95,$94.95,False,4.5 out of 5,60
4,Instinct VS Climbing Shoes,Scarpa,$141.69,$189.00,True,4.5 out of 5,23
5,Origin Climbing Shoes,Scarpa,$74.19,$99.00,True,NaN,0
6,Focus Climbing Shoes,Black Diamond,$99.93,$179.95,True,3.8 out of 5,6
7,Momentum Climbing Shoes,Black Diamond,$46.73,$94.95,True,4.5 out of 5,4
8,Free Range Climbing Shoes,So iLL,$80.73,$149.00,True,4.2 out of 5,24
9,Arpia Climbing Shoes,Scarpa,$119.19,$159.00,True,5 out of 5,3


### Step 2: Connect to an SQL database
